In [5]:
# Github Education
# Github Copilot

import gurobipy as gp
from gurobipy import GRB

In [6]:
class Determinant:
    def __init__(self, row1:list, row2:list):
        self.row1 = row1
        self.row2 = row2
        return None
    def getDeterminant(self):
        return self.row1[0] * self.row2[1] - self.row1[1] * self.row2[0]

In [7]:
class Point:
    def __init__(self, x, y):
        self.xCoordinate = x
        self.yCoordinate = y
        return None
    def compareTwoPoints(self, another):
        if self.xCoordinate > another.xCoordinate:
            return "greater"
        if self.xCoordinate < another.xCoordinate:
            return "less"
        else:
            if self.yCoordinate > another.yCoordinate:
                return "greater"
            if self.yCoordinate < another.yCoordinate:
                return "less"
            else:
                return "equal"

In [8]:
class Segment:
    def __init__(self, point1:Point, point2:Point):
        self.point1 = point1
        self.point2 = point2
        return None
    def IsIntersect(self, another):
        # Return True if two segments intersect inside.
        # Return False if two segments do not intersect inside.
        FirstEquation = [self.point2.xCoordinate - self.point1.xCoordinate, another.point1.xCoordinate - another.point2.xCoordinate, another.point1.xCoordinate - self.point1.xCoordinate]
        SecondEquation = [self.point2.yCoordinate - self.point1.yCoordinate, another.point1.yCoordinate - another.point2.yCoordinate, another.point1.yCoordinate - self.point1.yCoordinate]
        determinant = Determinant([FirstEquation[0], FirstEquation[1]], [SecondEquation[0], SecondEquation[1]]).getDeterminant()
        determinantX = Determinant([FirstEquation[2], FirstEquation[1]], [SecondEquation[2], SecondEquation[1]]).getDeterminant()
        determinantY = Determinant([FirstEquation[0], FirstEquation[2]], [SecondEquation[0], SecondEquation[2]]).getDeterminant()
        if determinant == 0:
            if determinantX == 0 and determinantY == 0:
                return True
            else:
                return False
        else:
            parameter1 = determinantX / determinant
            parameter2 = determinantY / determinant
            if parameter1 > 0 and parameter1 < 1 and parameter2 > 0 and parameter2 < 1:
                return True
            else:
                return False
    def IsStartPoint(self, point:Point):
        if point.compareTwoPoints(self.point1) == "equal" or point.compareTwoPoints(self.point2) == "equal":
            return True
        else:
            return False

In [9]:
# Testing whether class Point and class Segment are hashable. 
x = {}
Point1 = Point(0, 0)
Point2 = Point(1, 1)
Point3 = Point(2, 2)
Point4 = Point(3, 3)
Segment1 = Segment(Point1, Point2)
x[Segment1] = 1
print(x)

{<__main__.Segment object at 0x0000022279BEB350>: 1}


In [10]:
class HashiPuzzle:
    def __init__(self, listPoints:list, goalOrderList:list):
        self.listPoints = listPoints
        self.goalOrderList = goalOrderList
        self.model = gp.Model("HashiProblem")
    def getAdjacentPoints(self, index):
        # Return a list of adjacent points of point.
        point = self.listPoints[index]
        sameRowPointstoLeftList = [x for x in self.listPoints if x.xCoordinate == point.xCoordinate and x.yCoordinate < point.yCoordinate]
        sameRowPointstoRightList = [x for x in self.listPoints if x.xCoordinate == point.xCoordinate and x.yCoordinate > point.yCoordinate]
        sameColumnPointstoUpList = [x for x in self.listPoints if x.yCoordinate == point.yCoordinate and x.xCoordinate < point.xCoordinate]
        sameColumnPointstoDownList = [x for x in self.listPoints if x.yCoordinate == point.yCoordinate and x.xCoordinate > point.xCoordinate]
        adjacentPointstoLeftList = []
        adjacentPointstoRightList = []
        adjacentPointstoUpList = []
        adjacentPointstoDownList = []
        if len(sameRowPointstoLeftList) != 0:
            adjacentPointstoLeftList.append(max(sameRowPointstoLeftList, key = lambda x: x.yCoordinate))
        if len(sameRowPointstoRightList) != 0:
            adjacentPointstoRightList.append(min(sameRowPointstoRightList, key = lambda x: x.yCoordinate))
        if len(sameColumnPointstoUpList) != 0:
            adjacentPointstoUpList.append(max(sameColumnPointstoUpList, key = lambda x: x.xCoordinate))
        if len(sameColumnPointstoDownList) != 0:
            adjacentPointstoDownList.append(min(sameColumnPointstoDownList, key = lambda x: x.xCoordinate))
        adjacentPointsList = adjacentPointstoLeftList + adjacentPointstoRightList + adjacentPointstoUpList + adjacentPointstoDownList
        adjacentPointsIndexList = [self.listPoints.index(x) for x in adjacentPointsList]
        return adjacentPointsIndexList
    def getPotentialEdges(self):
        # Return a list of potential edges.
        potentialEdges = []
        for i in range(len(self.listPoints)):
            for j in self.getAdjacentPoints(i):
                if self.listPoints[i].compareTwoPoints(self.listPoints[j]) == "less":
                    potentialEdges.append(Segment(self.listPoints[i], self.listPoints[j]))
        return potentialEdges
    def getAllPotentialIntersectionPairs(self):
        # Return a list of all pairs of potential edges that intersect inside.
        potentialEdges = self.getPotentialEdges()
        for i in range(len(potentialEdges)):
            for j in range(i + 1, len(potentialEdges)):
                if potentialEdges[i].IsIntersect(potentialEdges[j]):
                    potentialEdges.append((potentialEdges[i], potentialEdges[j]))
    def setIsDirectlyConnectedVariables(self):
        self.x = {edge: self.model.addVar(vtype=GRB.BINARY, name = f'x_{self.listPoints.index(edge.point1)}_{self.listPoints.index(edge.point2)}')
                for edge in self.getPotentialEdges()}
    def setNumberOfBridgesVariables(self):
        self.y = {edge: self.model.addVar(vtype=GRB.INTEGER, name = f'y_{self.listPoints.index(edge.point1)}_{self.listPoints.index(edge.point2)}', ub = 2, lb = 0)
                for edge in self.getPotentialEdges()}
        self.relationBetweenXandY = [[self.model.addConstr(self.y[edge] <= 2 * self.x[edge]), self.model.addConstr(self.y[edge] >= self.x[edge])] for edge in self.x]
    def setDegreeConstraint(self):
        self.degreeConstraint = [self.model.addConstr(sum([self.y[edge] for edge in self.getAllPotentialIntersectionPairs()
                                                           if edge.IsStartPoint(point) == True]) == self.goalOrderList[self.listPoints.index(point)])
                                 for point in self.listPoints]
    def setNoIntersectionConstraint(self):
        # Set the constraint that no two bridges intersect inside.
        self.noIntersectionConstraint = [self.model.addConstraint(sum([self.x[pair[0]], self.x[pair[1]]]) <= 1)
                                         for pair in self.getAllPotentialIntersectionPairs()]
    def CheckTheConnectivity(self):
        # A graph is connected if and only if for every i, there exists a path from vertex 0 to vertex i.
        dictPairs = [(self.listPoints[0], self.listPoints[i]) for i in range(1, len(self.listPoints))]
        self.IsConnect = {pair: self.model.addVar(vtype=GRB.BINARY, name = f'IsConnect_{self.listPoints.index(pair[0])}_{self.listPoints.index(pair[1])}') for pair in dictPairs}
        
    def solve(self):
        self.setIsDirectlyConnectedVariables()
        self.setNumberOfBridgesVariables()
        self.setDegreeConstraint()
        self.setNoIntersectionConstraint()
        self.CheckTheConnectivity()
        self.model.optimize()
        return None